In [1]:
!pip install pandas
!pip install dask

Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 12.3 MB 52.7 MB/s eta 0:00:01
     |████████████████████████████████| 340 kB 33.7 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 33.3 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.5.0 requires numpy~=1.19.2, but you have numpy 1.24.2 which is incompatible.
deepfri 0.0.1 requires numpy==1.18.5, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://repo.huaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 1.2 MB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 42.5 MB/s eta 0:00:01
  

In [2]:
import pandas as pd
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import os
# Create a local cluster
# Get the number of physical cores
n_cores = os.cpu_count()
print(n_cores)
# Create a local cluster with one worker per core
cluster = LocalCluster(n_workers=int(n_cores/1.5), threads_per_worker=1)

# Connect to the cluster
client = Client(cluster)

# Your code here

import os
import csv

128


In [3]:
# os.chdir('./preprocessing')
alnRes = dd.read_csv('./data/alnRes.m8',sep='\t',header=None)
alnRes.columns = ['EntryID', 'col1', 'col2', 'col3', 'col4', 'col5', 'col6', 'col7', 'col8', 'col9', 'col10', 'col11']

alnRes.head(5)

,EntryID,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11
0,Q6ENB0,7f9o_Q,0.931,735,50,0,1,730,43,777,0.000000e+00,1354
1,Q6ENB0,7eu3_F,0.931,735,50,0,1,730,43,777,0.000000e+00,1354
2,Q6ENB0,7wg5_F,0.685,735,229,0,1,729,1,735,1.666977e-320,990
3,Q6ENB0,7wff_F,0.685,735,229,0,1,729,1,735,1.666977e-320,990
4,Q6ENB0,6khi_F,0.442,717,355,0,1,717,1,638,3.163000e-173,563


In [4]:
alnRes.compute().describe([.30])

,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11
count,1.378189e+07,1.378189e+07,1.378189e+07,13781891.0,1.378189e+07,1.378189e+07,1.378189e+07,1.378189e+07,1.378189e+07,1.378189e+07
mean,4.587226e-01,2.839119e+02,1.482073e+02,0.0,1.857246e+02,4.644306e+02,5.242068e+01,3.282708e+02,1.286453e-05,2.293885e+02
std,2.129429e-01,2.499623e+02,1.467580e+02,0.0,5.471455e+02,5.928421e+02,1.743324e+02,3.059957e+02,7.926572e-05,2.929405e+02
min,1.650000e-01,1.300000e+01,0.000000e+00,0.0,1.000000e+00,1.500000e+01,1.000000e+00,1.500000e+01,0.000000e+00,3.600000e+01
30%,3.190000e-01,1.600000e+02,6.800000e+01,0.0,1.700000e+01,2.480000e+02,4.000000e+00,1.820000e+02,6.822000e-65,9.000000e+01
50%,3.830000e-01,2.410000e+02,1.240000e+02,0.0,4.600000e+01,3.560000e+02,1.100000e+01,2.750000e+02,6.366000e-36,1.460000e+02
max,1.000000e+00,5.332000e+03,3.559000e+03,0.0,3.527700e+04,3.537500e+04,4.865000e+03,5.148000e+03,9.999000e-04,1.046800e+04


In [5]:
alnRes_clu = alnRes.col1.unique().compute().size
print("add clusters:",alnRes_clu)
alnRes_entry = alnRes.EntryID.unique().compute().size
print("Entry:",alnRes_entry)

add clusters: 570450
Entry: 112542


In [6]:
# Group the data by the id column
result = alnRes[alnRes['col10']<1e-6]
grouped = result.groupby('col1')

# Define a function to get the two rows with the smallest values in column 10 for each group
def get_smallest_two(group):
    return group.nsmallest(5, 'col10')
print('fliter')
# Apply the function to each group

result = grouped.apply(get_smallest_two, meta=result)

# Reset the index
result = result.compute()

result = result.reset_index(drop=True)
# Display the result
# print(result)
result.describe([.30])

fliter


2023-06-18 05:06:13,601 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 651.45 MiB -- Worker memory limit: 1.06 GiB
2023-06-18 05:10:52,034 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 797.70 MiB -- Worker memory limit: 1.06 GiB
2023-06-18 05:10:53,941 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. 

,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11
count,2.220585e+06,2.220585e+06,2.220585e+06,2220585.0,2.220585e+06,2.220585e+06,2.220585e+06,2.220585e+06,2.220585e+06,2.220585e+06
mean,7.007455e-01,3.004712e+02,9.067192e+01,0.0,1.033493e+02,4.004274e+02,1.504174e+01,3.106554e+02,3.904581e-09,4.083837e+02
std,2.617984e-01,2.923543e+02,1.436927e+02,0.0,4.753233e+02,5.553167e+02,5.545754e+01,2.952613e+02,4.466493e-08,4.350916e+02
min,1.770000e-01,1.800000e+01,0.000000e+00,0.0,1.000000e+00,2.100000e+01,1.000000e+00,1.800000e+01,0.000000e+00,4.500000e+01
30%,4.930000e-01,1.540000e+02,9.000000e+00,0.0,2.000000e+00,1.980000e+02,1.000000e+00,1.620000e+02,4.559000e-144,1.760000e+02
50%,7.360000e-01,2.400000e+02,4.700000e+01,0.0,1.400000e+01,2.970000e+02,3.000000e+00,2.490000e+02,4.742000e-82,2.770000e+02
max,1.000000e+00,5.332000e+03,3.559000e+03,0.0,3.527700e+04,3.537500e+04,4.334000e+03,5.148000e+03,9.988000e-07,1.046800e+04


In [7]:
result.to_csv('./data/alnRes_5byCLU.csv')

In [8]:
filter_clu = result.col1.unique().size
print("add clusters:",filter_clu)
print('del clusters:',alnRes_clu-filter_clu)
filter_entry = result.EntryID.unique().size
print("Entry:",filter_entry)
print('del Entry:',alnRes_entry-filter_entry)

add clusters: 560066
del clusters: 10384
Entry: 70669
del Entry: 41873


In [15]:
result2 = alnRes[alnRes['col10']<=1e-60]
result2.compute().describe([.30])

2023-06-18 04:46:52,408 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 804.34 MiB -- Worker memory limit: 1.06 GiB
2023-06-18 04:46:52,491 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 806.05 MiB -- Worker memory limit: 1.06 GiB
2023-06-18 04:46:52,573 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 869.71 MiB -- Worker memory limit: 1.06 GiB
2023-06-18 04:46:52,574 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memo

,col2,col3,col4,col5,col6,col7,col8,col9,col10,col11
count,4.429860e+06,4.429860e+06,4.429860e+06,4429860.0,4.429860e+06,4.429860e+06,4.429860e+06,4.429860e+06,4.429860e+06,4.429860e+06
mean,6.287138e-01,4.303339e+02,1.762776e+02,0.0,1.196980e+02,5.446187e+02,3.089267e+01,4.528418e+02,6.994112e-63,4.803307e+02
std,2.296400e-01,3.531053e+02,2.143038e+02,0.0,3.997497e+02,5.335888e+02,1.406221e+02,3.862059e+02,5.869817e-62,4.109411e+02
min,1.910000e-01,8.400000e+01,0.000000e+00,0.0,1.000000e+00,8.400000e+01,1.000000e+00,8.400000e+01,0.000000e+00,2.050000e+02
30%,4.550000e-01,2.780000e+02,5.900000e+01,0.0,4.000000e+00,3.290000e+02,2.000000e+00,2.880000e+02,1.752000e-153,2.890000e+02
50%,5.800000e-01,3.510000e+02,1.340000e+02,0.0,2.200000e+01,4.280000e+02,5.000000e+00,3.630000e+02,6.183000e-109,3.650000e+02
max,1.000000e+00,5.332000e+03,3.559000e+03,0.0,3.314700e+04,3.356700e+04,4.714000e+03,5.148000e+03,1.000000e-60,1.046800e+04


In [6]:
# result = pd.read_csv('./data/alnRes_2byCLU.csv',sep='\t')
# result.describe()

In [7]:
fluster_clu = result2.col1.unique().compute().size
print("fluster_clus:",fluster_clu)
fluster_entry = result2.EntryID.unique().compute().size
print("fluster_Entry:",fluster_entry)

fluster_clus: 405560
fluster_Entry: 70827


In [8]:
print('del clusters:',alnRes_clu-fluster_clu)
print('del Entry:',alnRes_entry-fluster_entry)

del clusters: 164890
del Entry: 41715


In [9]:
chain_go = pd.read_csv('./data/train_terms.tsv',sep='\t')
print('add GO terms',chain_go.term.unique().size)

add GO terms 31466


In [11]:
train_chain_go = pd.merge(result[['EntryID','col1']], chain_go[['EntryID', 'term']],how='left')

In [ ]:
old_data = pd.read_csv('./data/pdb_chain_go.tsv',sep='\t',skiprows=1)
old_data.head(5)

,PDB,CHAIN,SP_PRIMARY,WITH_STRING,EVIDENCE,GO_ID
0,101m,A,P02185,UniProtKB-KW:KW-0479,IEA,GO:0046872
1,101m,A,P02185,UniProtKB-KW:KW-0561,IEA,GO:0005344
2,101m,A,P02185,UniProtKB-KW:KW-0561,IEA,GO:0015671
3,102m,A,P02185,UniProtKB-KW:KW-0479,IEA,GO:0046872
4,102m,A,P02185,UniProtKB-KW:KW-0561,IEA,GO:0005344


In [12]:
_ = old_data.PDB + old_data.CHAIN
print('original clus:',_.unique().size)

original clus: 418330


In [13]:
print('add clus from fluster_clu:' ,len(set(result.col1.unique().compute()).difference(_.unique())))
print('more clus from alnRes:' ,len(set(alnRes.col1.unique().compute()).difference(_.unique())))

AttributeError: 'DataFrame' object has no attribute 'unique'

In [15]:
new_data = pd.DataFrame({
    'PDB': train_chain_go['col1'].str.split('_').str[0],
    'CHAIN': train_chain_go['col1'].str.split('_').str[1],
    'SP_PRIMARY': train_chain_go['EntryID'],
    'WITH_STRING': 'Kaggle',
    'EVIDENCE': 'train',
    'GO_ID': train_chain_go['term']
})

# Write the new DataFrame to a TSV file
new_data.head(5)

,PDB,CHAIN,SP_PRIMARY,WITH_STRING,EVIDENCE,GO_ID
0,11gs,B,V9HWE9,Kaggle,train,GO:0005829
1,11gs,B,V9HWE9,Kaggle,train,GO:0005622
2,11gs,B,V9HWE9,Kaggle,train,GO:0043229
3,11gs,B,V9HWE9,Kaggle,train,GO:0043226
4,11gs,B,V9HWE9,Kaggle,train,GO:0110165


In [16]:
pdb_chain_go_new = pd.concat([old_data,new_data], axis=0)
pdb_chain_go_new.to_csv('./data/pdb_chain_go_new_5byclu.tsv',index=False,sep='\t')